In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from PIL import Image

In [4]:
from google.colab import drive
drive.mount('/content/drive')
import os

Mounted at /content/drive


In [5]:
link = '/content/drive/MyDrive/Ativities/NextGenAI Camp/Onsite/Hackathon/dataset-nextgen-Day2'
train = pd.read_csv(os.path.join(link, 'train.csv'))
pre_final = pd.read_csv(os.path.join(link, 'test.csv'))
example = pd.read_csv(os.path.join(link, 'sample_submission_engagement_estimation.csv'))

In [6]:
train = train.drop(columns=['Hashtags', 'Id'])
Id_final = pre_final.Id
pre_final = pre_final.drop(columns=['Hashtags', 'Id'])

In [7]:
def name(df):
  df = df.rename(columns = {
      'FoodName' : 'Class',
      'CaptionLength' : 'Caption',
      'ImageQuality' : 'MOS',
      'Segment' : 'Seg',
      'TimePost' : 'Time',
  })
  return df

def season(month):
  if 2 <= month <= 5:
    return 'summer'
  elif 6 <= month <= 10:
    return 'rainy'
  elif 11 <= month <= 12 or 1 == month:
    return 'winter'

def prepare(df):
  df = name(df)
  df = name(df)
  df['Time'] = pd.to_datetime(df.Time)
  df['Month'] = df['Time'].dt.month
  df['Season'] = df['Month'].apply(season)
  df = df.drop(columns=['Month', 'Time'])
  return df

In [8]:
train = prepare(train)
pre_final = prepare(pre_final)

In [9]:
link_D1 = '/content/drive/MyDrive/Ativities/NextGenAI Camp/Onsite/Hackathon/dataset-nextgen-Day1'
dir_pre_final = os.path.join(link_D1, 'test')

pre_final = pre_final.rename(columns={
    'MissingDataId' : 'Image'
})
pre_final['Image'] = pre_final['Image'].apply(lambda x: f'image_{int(x)}.jpg' if pd.notnull(x) else np.nan)

In [10]:
import json
label_to_index = json.load(open("label to index.json", "r"))
pre_final['Class'] = pre_final['Class'].map(label_to_index)

index_to_label = {}
for label_name, label_index in label_to_index.items():
  index_to_label[label_index] = label_name

In [11]:
pre_final.head()

,Image,Class,Caption,MOS,Seg,Platform,Season
0,image_11882.jpg,NaN,164,NaN,P07,twitter,rainy
1,NaN,0.0,78,0.8,P04,twitter,rainy
2,image_8126.jpg,NaN,33,NaN,P03,instagram,winter
3,image_660.jpg,NaN,75,NaN,P12,instagram,winter
4,image_2956.jpg,NaN,165,NaN,P07,instagram,winter


In [12]:
class CNN(nn.Module):
  def __init__(self, num_label):
    super(CNN, self).__init__()
    self.features = nn.Sequential(
        nn.Conv2d(3, 32, 3, stride=1, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(32, 64, 3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(64, 128, 3, stride=1, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.AdaptiveAvgPool2d((7, 7))
    )

    self.fc = nn.Sequential(
        nn.Flatten(),
        nn.Linear(128 * 7 * 7, 256),
        nn.ReLU(),
        nn.Dropout(0.3)
    )

    self.regression = nn.Linear(256, 1) # MOS
    self.classification = nn.Linear(256, num_label) # label

  def forward(self, x):
    x = self.features(x)
    x = self.fc(x)
    mos = self.regression(x).squeeze(1)
    label = self.classification(x)
    return mos, label

In [13]:
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], # ImageNet
                [0.229, 0.224, 0.225])
])

In [14]:
temp = []
for i in pre_final['Image'] :
  if i is np.nan:
    temp.append('0')
  else :
    temp.append(os.path.join(dir_pre_final, i))
temp = pd.DataFrame(temp, columns=['Image'])
temp['Image'] = temp['Image'].replace('0', np.nan)
pre_final['Image'] = temp['Image']

In [15]:
num_label = len(label_to_index)
model = CNN(num_label)

x = torch.load('CNN_model.pth', map_location='cpu')
model.load_state_dict(x['model_state'])

temp = pre_final.drop(columns=['Caption', 'MOS', 'Seg', 'Platform', 'Season'])
df = pd.DataFrame()
model.eval()
for index, row in temp[temp['Class'].isna()].iterrows():
  img_path = row['Image']
  img = Image.open(img_path)
  img = transform(img).unsqueeze(0)

  with torch.no_grad():
    mos_hat, class_hat = model(img)

  mos_hat = mos_hat.item()
  class_hat = torch.argmax(class_hat, dim=1).item()
  df.loc[index, ['MOS', 'Class']] = [mos_hat, class_hat]

df = pd.DataFrame(df)
df

,MOS,Class
0,0.536196,43
2,0.554211,29
3,0.559219,29
4,0.55659,5
5,0.55172,29
...,...,...
2245,0.554359,36
2246,0.559492,29
2247,0.547685,5
2248,0.557792,29


In [16]:
pre_final.update(df)

/tmp/ipython-input-526968212.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[43 0.0 29 ... 5 29 5]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  pre_final.update(df)
/tmp/ipython-input-526968212.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.5361962914466858 0.8 0.5542105436325073 ... 0.5476853847503662
 0.5577921271324158 0.5504947900772095]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  pre_final.update(df)


In [17]:
pre_final = pre_final.drop(columns=['Image'])
pre_final.head()

,Class,Caption,MOS,Seg,Platform,Season
0,43,164,0.536196,P07,twitter,rainy
1,0.0,78,0.8,P04,twitter,rainy
2,29,33,0.554211,P03,instagram,winter
3,29,75,0.559219,P12,instagram,winter
4,5,165,0.55659,P07,instagram,winter


In [18]:
dummy = pd.get_dummies(pre_final[['Platform', 'Season', 'Seg']], dtype=int)
final = pd.concat([pre_final, dummy], axis=1)
final = final.drop(columns=['Seg', 'Platform', 'Season'])
final.head()

,Class,Caption,MOS,Platform_facebook,Platform_instagram,Platform_tiktok,Platform_twitter,Platform_youtube,Season_rainy,Season_summer,...,Seg_P11,Seg_P12,Seg_P13,Seg_P14,Seg_P15,Seg_P16,Seg_P17,Seg_P18,Seg_P19,Seg_P20
0,43,164,0.536196,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,78,0.8,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,29,33,0.554211,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,29,75,0.559219,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,5,165,0.55659,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
train['Class'] = train['Class'].map(label_to_index)
dummy = pd.get_dummies(train[['Platform', 'Season', 'Seg']], dtype=int)
train = pd.concat([train, dummy], axis=1)
train = train.drop(columns=['Platform', 'Season', 'Seg'])
y_train = train.Engagement
x_train = train.drop(columns=['Engagement'])
x_train.head()

,Class,Caption,MOS,Platform_facebook,Platform_instagram,Platform_tiktok,Platform_twitter,Platform_youtube,Season_rainy,Season_summer,...,Seg_P11,Seg_P12,Seg_P13,Seg_P14,Seg_P15,Seg_P16,Seg_P17,Seg_P18,Seg_P19,Seg_P20
0,3,26,0.22,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,9,6,0.55,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12,1,0.62,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,10,91,0.44,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,47,96,0.97,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [20]:
y_train.info()

<class 'pandas.core.series.Series'>
RangeIndex: 12750 entries, 0 to 12749
Series name: Engagement
Non-Null Count  Dtype  
--------------  -----  
12750 non-null  float64
dtypes: float64(1)
memory usage: 99.7 KB


In [21]:
tensor_x_train = torch.tensor(x_train.values.astype(np.float32))
tensor_y_train = torch.tensor(y_train.values.astype(np.float32))
tensor_final = torch.tensor(final.values.astype(np.float32))

In [22]:
class LinearRegression(nn.Module):
  def __init__(self, num_label, output=1):
    super(LinearRegression, self).__init__()
    self.linear = nn.Linear(num_label, output)

  def forward(self, x):
    return self.linear(x)

num_label = x_train.shape[1]
model = LinearRegression(num_label)
lr = 0.001
epoch = 10_000

error = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
n = 1000
for i in range(0, epoch):
  model.train()
  y_hat = model(tensor_x_train)
  MSE_loss = error(y_hat, tensor_y_train)

  optimizer.zero_grad()
  MSE_loss.backward()
  optimizer.step()

  if i+1 == n:
    print(f"epoch : {i+1} | MSE : {MSE_loss}")
    n += 1000

torch.save({'model_state': model.state_dict()}, 'LinearRegression_model.pth')

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([12750])) that is different to the input size (torch.Size([12750, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch : 1000 | MSE : 127985.984375
epoch : 2000 | MSE : 99500.09375
epoch : 3000 | MSE : 90634.1875
epoch : 4000 | MSE : 88474.7109375
epoch : 5000 | MSE : 87151.8984375
epoch : 6000 | MSE : 85852.421875
epoch : 7000 | MSE : 84749.8828125
epoch : 8000 | MSE : 83942.015625
epoch : 9000 | MSE : 83388.6796875
epoch : 10000 | MSE : 82966.78125


In [23]:
model.eval()
with torch.no_grad():
    y_hat = model(tensor_final)
summarize = pd.concat([Id_final, (pd.DataFrame(y_hat.numpy(), columns=['Predicted_Engagement']))], axis=1)
summarize.to_csv('My_Answer.csv', index=False)

In [24]:
summarize

,Id,Predicted_Engagement
0,81d4d808-427c-4bb6-beb9-e49c11318ad8,481.733582
1,12666922-65ab-4adc-aa5c-f70a553c7056,143.055878
2,0727c597-156f-417a-bddf-ae8f58cacbbe,232.557175
3,1eacd229-1a0f-443f-9a15-b545838eed6a,289.328522
4,3e2b023c-ede1-4e3a-be66-139f2405a25b,285.175110
...,...,...
2245,deaaf745-a72b-4616-9037-de7f5b6680d5,300.009735
2246,2f79ce2f-aea2-4a57-9eef-0b5df1094893,234.709000
2247,8c3b1b01-1940-403e-b94a-914a00e8a609,238.733231
2248,b4b6ab9b-6bfa-4ac1-a122-557b0f9a467b,389.116028
